In [1]:
import os
import sys
from glob import glob

os.chdir('..') if not os.getcwd().endswith('data') else {}

In [3]:
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from utils.pipeline_utils import *
from tqdm.notebook import tqdm
from utils.general_utils import normalized_col

pd.set_option('display.max_columns', None)

In [4]:
pipeline_path = f'{run_prefix}pipeline'
pred_path = f'{pipeline_path}/csv/merged/*.csv'
files = glob(pred_path)
analyze_tweets_dir = f'{run_prefix}data/analyze_tweets'
os.makedirs(f'{analyze_tweets_dir}/csv', exist_ok=True)
os.makedirs(f'{analyze_tweets_dir}/figs', exist_ok=True)

Load Job offer Data

In [5]:
merged_df = pd.concat([pd.read_csv(f, usecols=['created_at', 'category', 'category_label', 'address'], parse_dates=['created_at']) for f in tqdm(files)])

  0%|          | 0/191 [00:00<?, ?it/s]

In [6]:
merged_df['state'] = merged_df.address.apply(lambda add: add.split(', ')[-2])
merged_df = merged_df.rename(columns={'created_at': 'date'})
merged_df = merged_df.set_index('date').drop(['address'], axis=1)
print(f"Total Number os samples: {len(merged_df)}")

Total Number os samples: 23172957


Load dedup Data

In [7]:
tweet_counts = pd.read_csv(f'{run_prefix}data/job_offer/dedup_counts.csv', parse_dates=['date'])[['date', 'total_tweets']]
tweet_counts.columns = ['date', 'Twitter']
tweet_counts = tweet_counts.set_index('date')

Load BGT Data

In [8]:
bgt_data = pd.read_csv(f'{run_prefix}data/job_offer/jolt data/BGT Data.csv')
bgt_data['date'] = pd.to_datetime(bgt_data['date'])
bgt_data = bgt_data.set_index('date')

Get categories mapping

In [9]:
cat_map = bgt_data[['category', 'category_label']].drop_duplicates().set_index('category')
cat_map.loc[0] = pd.Series({'category_label': 'Armed Forces Occupations'})

In [10]:
cat_map

,category_label
category,
1,Managers
2,Professionals
3,Technicians and associate professionals
4,Clerical support workers
5,Service and sales workers
6,"Skilled agricultural, forestry and fishery wor..."
7,Craft and related trades workers
8,"Plant and machine operators, and assemblers"
9,Elementary occupations


In [11]:
merged_df['category'].value_counts().sort_index()

0.0     119842
1.0    3652040
2.0    9690751
3.0    3408124
4.0    1971954
5.0    1807461
6.0      79088
7.0    1235757
8.0     696997
9.0     510943
Name: category, dtype: int64

In [12]:
if 'sub_major' in pred_path:
    merged_df['category'] = merged_df['category'] // 10
    merged_df['category_label'] = cat_map.loc[merged_df['category']].values

Load Employment Data

In [13]:
employment_data = pd.read_csv(f'{run_prefix}data/job_offer/jolt data/Employment Data.csv')
employment_data['date'] = pd.to_datetime(employment_data['date'])
employment_data = employment_data.set_index('date')

Load Jolt Data

In [14]:
jolt_data = pd.read_csv(f'{run_prefix}data/job_offer/jolt data/jolt_data.csv')
jolt_data['date'] = pd.to_datetime(jolt_data['year'].map(str) + '-' + jolt_data['month'].map(str)) + \
                    pd.tseries.offsets.MonthEnd(0)
jolt_data = jolt_data.set_index('date')

Twitter Inference - monthly/states

In [15]:
state_twitter_jolt_monthly = jolt_data.query("state!='Total US'").groupby(['state', 'date']).sum()[
    ['JOLTS Job openings', 'JOLTS Hires', 'JOLTS Separations']] * 1000
state_twitter_jolt_monthly['Twitter'] = merged_df.groupby('state').resample('M').count()['category']
state_twitter_jolt_monthly = state_twitter_jolt_monthly.dropna()
state_twitter_jolt_monthly.to_csv(f'{analyze_tweets_dir}/csv/state_twitter_jolt_monthly.csv')

In [16]:
state_twitter_jolt_monthly_corr = state_twitter_jolt_monthly.corr()  # [['JOLTS Job openings', 'Twitter']]
state_twitter_jolt_monthly_corr

,JOLTS Job openings,JOLTS Hires,JOLTS Separations,Twitter
JOLTS Job openings,1.000000,0.935863,0.888281,0.561365
JOLTS Hires,0.935863,1.000000,0.939145,0.691260
JOLTS Separations,0.888281,0.939145,1.000000,0.669025
Twitter,0.561365,0.691260,0.669025,1.000000


Twitter (Inference and dedup) and BGT share of Employment - yearly

In [17]:
us_twitter_bgt_emp_yearly =  employment_data.query("category_label!='Total'").groupby('category').resample('Y').sum()[['employment']].\
    rename({'employment': 'Employment'}, axis=1)
us_twitter_bgt_emp_yearly['Twitter'] = merged_df.groupby('category').resample('Y').count()[['category_label']]
us_twitter_bgt_emp_yearly['BGT'] = bgt_data.query("category_label!='Total'").groupby('category').resample('Y').sum()['BGT']

In [18]:
dfs = []
for _, df in us_twitter_bgt_emp_yearly.groupby('date'):
    df['Twitter normalized by year'] = normalized_col(df, 'Twitter', 'Employment')
    df['BGT normalized by year'] = normalized_col(df, 'BGT', 'Employment')
    dfs.append(df)
us_twitter_bgt_emp_yearly = pd.concat(dfs)

In [19]:
dfs = []
for _, df in us_twitter_bgt_emp_yearly.groupby('category'):
    df['Twitter normalized by category'] = normalized_col(df, 'Twitter', 'Employment')
    df['BGT normalized by category'] = normalized_col(df, 'BGT', 'Employment')
    dfs.append(df)
us_twitter_bgt_emp_yearly = pd.concat(dfs)

In [20]:
us_twitter_bgt_emp_yearly = us_twitter_bgt_emp_yearly.reset_index()
us_twitter_bgt_emp_yearly['Category Label'] = cat_map.loc[us_twitter_bgt_emp_yearly['category']].reset_index().category_label
us_twitter_bgt_emp_yearly.to_csv(f'{analyze_tweets_dir}/csv/us_twitter_bgt_emp_yearly.csv', index=False)

In [21]:
us_twitter_bgt_emp_yearly_corr = us_twitter_bgt_emp_yearly.corr()  # [['Twitter', 'BGT', 'Employment']]
us_twitter_bgt_emp_yearly_corr

,category,Employment,Twitter,BGT,Twitter normalized by year,BGT normalized by year,Twitter normalized by category,BGT normalized by category
category,1.000000e+00,-0.265855,-0.613783,-0.618581,-0.615883,-0.654500,1.828005e-17,-1.626275e-17
Employment,-2.658548e-01,1.000000,0.397767,0.650402,-0.112878,-0.070316,3.904318e-03,5.694561e-02
Twitter,-6.137833e-01,0.397767,1.000000,0.706061,0.741699,0.708387,3.517561e-01,-2.719039e-02
BGT,-6.185810e-01,0.650402,0.706061,1.000000,0.552116,0.602741,-5.881604e-02,2.785601e-01
Twitter normalized by year,-6.158833e-01,-0.112878,0.741699,0.552116,1.000000,0.964034,4.828952e-02,2.816506e-02
BGT normalized by year,-6.545000e-01,-0.070316,0.708387,0.602741,0.964034,1.000000,4.826147e-02,7.250885e-02
Twitter normalized by category,1.828005e-17,0.003904,0.351756,-0.058816,0.048290,0.048261,1.000000e+00,2.541677e-02
BGT normalized by category,-1.626275e-17,0.056946,-0.027190,0.278560,0.028165,0.072509,2.541677e-02,1.000000e+00
